# This Notebook aims on clustering of neighborhoods in toronto. Therefore, the data first has to be scraped from a Wikipedia website. 

## Before we can begin with the coding, we need to import all relevant libraries:

In [5]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

## Importing relevant data and converting it into an usable dataframe

Now we can start with downloading the table from the website. Afterwards the html information is parsed by the **BeautifulSoup** method.

In [16]:
#This code is used for downloadingt he table from the website and parsing it
website_info = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(website_info.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

The next step is to interpret the parsed html source-code and select the relevant item from it. Following this, we can slowly rebuild the table from the website into a **Pandas Dataframe**. To understand the following code some basic understanding about html code is necessarry: 

In HTML code, when defining a table the syntax uses the <*table*> tag. Furthermore, each table row is defined with the <*tr*> tag. A table header is defined with the <*th*> tag. By default, table headings are bold and centered. A table data/cell is defined with the <*td*> tag. Hence, we start with finding all **HEADER** colummns through searching then by their tag with the python .findALL command. Afterwards, we clean up the html code by removing the tag markers and commands for a new line (\\n).  

Then we do the same for all the rows using .findALL command with <*tr*> as input representing the table rows. Similar to the **HEADERs**, we remove all symbols which are used for HTML code formulation but unnecessary for our DataFrame. We must make sure to skip the first row (index = 0) so that we dont have the header's row twice. 

In [17]:
#get headers:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

#Find all items and skip first one:
rows=table.findAll('tr')
rows=rows[1:len(rows)]
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","").replace("\n","")

Now we can create our data frame with the *pd.DataFrame* command.   
This creates the basic data frame. However, all the information is combined in just one column, separated by html code (</*td*>\n<*td*>). Lets take a look at it: 

In [18]:
df=pd.DataFrame(rows)
df.head()

,0
0,M1A</td><td>Not assigned</td><td>
1,M2A</td><td>Not assigned</td><td>
2,M3A</td><td>North York</td><td>Parkwoods
3,M4A</td><td>North York</td><td>Victoria Village
4,M5A</td><td>Downtown Toronto</td><td>Regent Pa...


Consequently, we will have to put the information into separate columns. To do so, we can use the **pandas.Series.str.split** command.  
In general, this command looks like this: *Series.str.split(self, pat=None, n=-1, expand=False)* and what it does is that it splits strings around a given separator.   

The input parameters are:  
>*patstr, optional*                  ==> String or regular expression to split on. If not specified, split on whitespace.  
>*n (as int), default -1 (all)*      ==> Limit number of splits in output. None, 0 and -1 will be interpreted as return all splits.  
>*expand(boolean, default=False)*    ==> Expand the splitted strings into separate columns.  
                                     If **True**, return DataFrame/MultiIndex expanding dimensionality.  
                                     If **False**, return Series/Index, containing lists of strings. 
                                       
The output is either a Series, Index, DataFrame or MultiIndex.   
  
For our problem, the expression to split on is the HTML code (</*td*><*td*>). As n we can either input 2 (since we already know the amount of rows we want this might be beneficial to avoid errors) or keep it on default setting. For the expand parameter we use **TRUE** because we want to split the colummn into multiple columns and receive a new data frame.

In [19]:
df[headers] = df[0].str.split("</td><td>", n = 2, expand = True) 
df[headers].head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Looking at our main data frame again, we notice that we have to drop the old column since we dont need it anymore. We use the parameter inplace=**TRUE** to maintain the same data frame.

In [20]:
df.head()

,0,Postal code,Borough,Neighborhood
0,M1A</td><td>Not assigned</td><td>,M1A,Not assigned,
1,M2A</td><td>Not assigned</td><td>,M2A,Not assigned,
2,M3A</td><td>North York</td><td>Parkwoods,M3A,North York,Parkwoods
3,M4A</td><td>North York</td><td>Victoria Village,M4A,North York,Victoria Village
4,M5A</td><td>Downtown Toronto</td><td>Regent Pa...,M5A,Downtown Toronto,Regent Park / Harbourfront


In [21]:
df.drop(columns=[0], inplace = True)
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Now we need tog get rid of all unassigned boroughs and give neighborhoods the name of their borough if not specified. Moreover, we drop all dublicate rows, reset the index so that it is continuous, and replace all / with , as required by the task definition. 

In [61]:
df = df.drop(df[(df.Borough == "Not assigned")].index)
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)
df=df.drop_duplicates()
df.reset_index(drop=True, inplace=True)
df['Neighborhood']= df['Neighborhood'].str.replace('/',',')
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


Finally we check the shape of the dataframe.

In [63]:
df.shape

(103, 3)

## Importing GEO-data and adding longitute and latitude coordinates to the data frame  
  
    
The next step is to import the geo-spatial data and merge it with the obtained data frame. For the sake of reliability the *.csv-file* is used. 



In [67]:
df_geo= pd.read_csv("http://cocl.us/Geospatial_data")


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


After reading the *.csv-file* and converting it into a data frame, we can prepare the two data frames for merging. As we merge the data frames by index, it makes sense to define the Postal code as index for each data frame so that the geo-spatial data matches the neighbourhoods. 

In [79]:
df_geo.rename(columns={'Postal Code':'Postal code'}, inplace=True)
df_geo.set_index("Postal code")
df.set_index("Postal code")
df_geo.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now, we can merge the two data frames:


In [80]:
toronto_data=pd.merge(df, df_geo)
toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


## Mapping Toronto and exploring the Neighborhoods  

In order to cluster the neighboorhoods we will start with mapping what we've found so far. Therefore, we use **geolocator** to find longitute and latitute coordinates of Toronto and afterwards draw a map with neighborhoods superimposed on it using **folium**.

In [85]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="explore_toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FFAA99',
        fill_opacity=0.8,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

So after we have briefly explored the city, we want to cluster the neighborhoods by their venues. Before we can do so, we will walk trough the following steps:  

>1. Receive neighborhood data through the foursquare API  
>2. Define a function that returns venues for each neighborhood in Toronto
>3. Analyze each neighborhood and group them by occurance of venue-types
>4. Define the most occuring venue-types for each neighborhood 

### Using the foursquare API to receive neighborhood data

First, we need to input our foursquare credentials:

In [88]:
CLIENT_ID = 'TEKN3ZX3NEVUYE0XFHPMZ3E4R0JXY2CJ5H0WLOMQSF4L5WSZ' # your Foursquare ID
CLIENT_SECRET = 'J233DJX4TM4DCROEBHIVYCHINPK2FEQWX2PRVNIJBHFCTHV1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

Now, we can define a function to return venues for each neighborhood in Toronto and let it run through all neighborhoods

In [89]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#Running the function
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Queen's Park , Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don Mills
Parkview Hill , Woodbine Gardens
Garden District , Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
Bayview Village
D

After the data selection through foursquare we can take a look at the shape of the resulting data frame and check how many unique venue categories there are:

In [90]:
print(toronto_venues.shape)
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

(2151, 7)
There are 267 uniques categories.


Now,we use the *pd.get_dummies* method to translate the appearance of a specific venue into numeric values so that the algorythm later on can work with it.  
Dont forget to insert the name of the neighborhood back to the data frame and put it at position one so that its easier to use.   
Finally, we can group the neighborhoods by taking the mean of the frequency of occurance of each venue category. 

In [91]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

To double-check that the data frame matches our amount of unique categorys we take a look at the *shape*:

In [92]:
print(toronto_grouped.shape)

(93, 267)


Now we want a data frame, where to top 10 for each neighborhood are displayed. Therefore, we start with a funtion to create said data frame.   
  
Looking at the *return_most_common_venues* function we see that it actually just places the venus for each row(=neighborhood) in descending order.



In [94]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now we can start to create the data frame: 


In [97]:
num_top_venues = 10 #Defines how many venues we observe for each neighborhood

indicators = ['st', 'nd', 'rd'] #Notation used, so that the column names are grammatically correct

# create columns according to number of top venues
columns = ['Neighborhood'] #First, we create a column where we place the neighborhoods in later on
for ind in np.arange(num_top_venues): #Now we create columns by using a numpy array with the lenght of 10 (number depending on num_top_venues input)
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) #Using the correct numbering for the 1st, 2nd and 3rd venue
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) #Using the correct numbering for 4th to 10th venue

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns) #Creating a data frame with column names as defined above
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood'] #importing the neighborhoods into the data frame

for ind in np.arange(toronto_grouped.shape[0]): #loop that goes through axis 0 (= neighborhood 0-93)
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues) #each row imports the most common venues until it has reached the num_top_venues

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Clothing Store,Breakfast Spot,Skating Rink,Event Space,Ethiopian Restaurant,Electronics Store,Dim Sum Restaurant,Eastern European Restaurant
1,"Alderwood , Long Branch",Pizza Place,Gym,Pub,Sandwich Place,Dance Studio,Athletics & Sports,Skating Rink,Coffee Shop,Pharmacy,Concert Hall
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Shopping Mall,Bridal Shop,Ice Cream Shop,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Fried Chicken Joint,Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park , Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant,Thai Restaurant,Pharmacy,Pizza Place,Cosmetics Shop,Pub


## Clustering usign k-means  

After the pre-processing of the data is finally done, we can use the k-means algorythm to create clusters according to the most common venues. 

In [132]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 3, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 0, 1, 2, 2, 2, 1,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1,
       2, 2, 2, 2, 1], dtype=int32)

To visualize our findings, we can display the labels of the neighbourhoods by merging them with prevous data frames

In [138]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace = True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)


# check the last columns!
toronto_merged

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Food & Drink Shop,Park,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Hockey Arena,Coffee Shop,Intersection,Portuguese Restaurant,French Restaurant,Pizza Place,Eastern European Restaurant,Electronics Store,Drugstore,Donut Shop
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Theater,Breakfast Spot,Café,Shoe Store,Restaurant,Chocolate Shop
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Coffee Shop,Boutique,Shoe Store,Sporting Goods Shop,Miscellaneous Shop,Arts & Crafts Store,Women's Store,Vietnamese Restaurant
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Burger Joint,Bank,Bar,Beer Bar,Italian Restaurant,Japanese Restaurant
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Deli / Bodega
7,M3B,North York,Don Mills,43.745906,-79.352188,1,Gym,Restaurant,Beer Store,Japanese Restaurant,Coffee Shop,Asian Restaurant,Concert Hall,Chinese Restaurant,Bike Shop,Italian Restaurant
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Pharmacy,Athletics & Sports,Gastropub,Intersection,Fast Food Restaurant,Pet Store,Breakfast Spot,Bank,Gym / Fitness Center
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Cosmetics Shop,Theater
10,M6B,North York,Glencairn,43.709577,-79.445073,1,Park,Japanese Restaurant,Pub,Metro Station,Pizza Place,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


# Visualization of neighborhood clusters  

Finally, we can create a map with all the clusters shown on the Toronot map we've created before. 

In [139]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters